# (체험) 제2유형

**유형: 프로그래밍**

---

## 문제 설명

제공된 학습용 데이터(customer_train.csv)를 활용하여 고객의 나전건 상품 구매 가능여부를 예측하는 데이터를 활용하여 총 구매예측을 해주는 모델을 개발하고, 이를 기초 주고받는 평가용 데이터셋(customer_test.csv)에서 적용하여 총구매예측을 예측하시오. 예측 결과는 아래의 [제출 형식]를 준수하여, CSV 파일로 저장하시오.

**예측 결과는 RMSE(Root Mean Squared Error) 평가지표로 마미 평가를 [제출 형식]**

---

## CSV 파일명: result.csv (파일형태에 따라테크크 정원기)

a. 예측 총 구매예측 컬럼명: pred  
b. 제출 컬럼 개수: pred 컬럼 1개  
c. 평가용 데이터 개수와 예측 결과 데이터 개수 일치: 2,482개

---

## 제공 데이터

### 데이터 특징

**customer_train.csv**: 학습용 데이터, 3,500개  
**customer_test.csv**: 평가용 데이터, 2,482개  
* 평가용 데이터는 총 구매예측 송출 미제시

### 데이터 설명

| 컬럼명 | 컬럼 설명 |
|--------|-----------|
| 직원ID | 직원 고유번호 |
| 종구매예측 | 총 구매예측(신만원) |
| 직장 구매예측 | 그동안 직장구매예측(신만원) |
| 한동금 | 한동금(신만원) |
| 주구매 상품 | 주로 구매는 품목(가구, 용품 변경) |
| 주구매 시장 | 주로 구매한 시장종류(A 사진 등급) |
| 반응점수 | 고객의 반응등 정도(점) |
| 방문점수반정도 | 총구매단/관련결과 |
| 서비스평점 | 서비스 평결결고 평균 |
| 고객등급 | 고객 등급(골) |

---

## CSV 파일 업로드 및 절차 방법

### CSV 파일명: result.csv

**예측 총 구매예측 전 컬럼명: pred**

| 컬럼명 | 컬럼 설명 |
|--------|-----------|
| pred | 예측 총 구매예측(신만원) |

---

## 제출 CSV 파일 일부 예시

### CSV 파일 예시

pred  
8790.6061268  
14.748660  
215.6035114  
761.88  
...  
458.1174502

---

## CSV 파일 불러 오기 및 상호 무결정 여제 예시 따라

**좋확하여 접근**

| Python 예시 | R 예시 |
|-------------|--------|
| import pandas as pd<br>result = pd.read_csv('result.csv')<br>print(result) | result = read.csv('result.csv')<br>print(result) |

In [1]:
# 제 2유형은 회귀인지 분류인지 파악하는 게 필요
# 1. 총구매액 예측 / 총구매액 : 연속형 숫자 -> 회귀
# 2. RMSE / E(에러) -> 회귀
# 3. 제출 CSV 파일 예시 / 소수점 : 연속형 숫자 -> 회귀

# 학습 데이터의 결측치인 환불금액을 어떻게 처리하냐?
# RMSE가 평가지표이므로 0, 중앙값, 평균값 넣어본 뒤 RMSE가 가장 작은 것으로 전처리

import pandas as pd

train = pd.read_csv("data/customer_train.csv")
test = pd.read_csv("data/customer_test.csv")

# EDA
pd.set_option('display.max_columns', None)

# null 개수 확인
print(train.isnull().sum())
print('---------------------------')
print(test.isnull().sum())
print('---------------------------')

# unique 개수 확인 (주구매상품의 unique가 다름)
print(train.describe(include='O'))
print('---------------------------')
print(test.describe(include='O'))
print('---------------------------')

# 어디에 무슨 컬럼이 더 있는 지 확인
print('train에만 있는 컬럼: ', set(train['주구매상품']) - set(test['주구매상품']))
print('---------------------------')
print('test에만 있는 컬럼: ', set(test['주구매상품']) - set(train['주구매상품']))
print('---------------------------')

# 전처리
target = train.pop('총구매액')

# null 0으로 채움
train['환불금액'] = train['환불금액'].fillna(0)
test['환불금액'] = test['환불금액'].fillna(0)

# case1) 레이블 인코딩
# RMSE:  806.7392576360363
from sklearn.preprocessing import LabelEncoder
cols = ['주구매상품', '주구매지점']
for col in cols:
	le = LabelEncoder()
	train[col] = le.fit_transform(train[col])
	test[col] = le.transform(test[col])

# case2) 원핫 인코딩 (카테고리가 다르면 합쳐서 진행 필요함)
# RMSE:  827.2106861445362
# df = pd.concat([train, test])
# df = pd.get_dummies(df)
# train = df.iloc[:len(train)] # 다시 분리
# test = df.iloc[len(train):]

# 검증용 데이터
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)
print('---------------------------')

# 모델 학습
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

# 모델 평가
from sklearn.metrics import root_mean_squared_error
print('RMSE: ', root_mean_squared_error(y_val, pred))
print('---------------------------')

# 제출
pred = model.predict(test)
result = pd.DataFrame({
	'pred': pred
})
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/customer_train.csv'